In [1]:
!pip install xgboost

     |████████████████████████████████| 192.9 MB 33.9 MB/s eta 0:00:01     |████████████████████████        | 144.5 MB 38.9 MB/s eta 0:00:02


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import xgboost as xgb
import sklearn
print("XGB Version          : ", xgb.__version__)
print("Scikit-Learn Version : ", sklearn.__version__)

XGB Version          :  1.6.1
Scikit-Learn Version :  0.23.2


In [3]:
# df = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/main/Leads.csv")
df = pd.read_csv("https://raw.githubusercontent.com/fenago/datasets/main/churn.csv")
df.sample(5)

,churn,accountlength,internationalplan,voicemailplan,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
4737,Yes,137,no,no,0,111.8,106,19.01,281.9,89,23.96,233.0,93,10.49,11.5,4,3.11,4
3104,No,63,no,yes,13,214.2,61,36.41,181.2,88,15.40,174.0,68,7.83,10.3,2,2.78,0
2805,No,166,yes,yes,28,175.8,126,29.89,253.6,76,21.56,128.5,72,5.78,11.4,5,3.08,1
4280,Yes,104,no,no,0,210.9,90,35.85,272.2,101,23.14,304.3,86,13.69,10.6,5,2.86,3
2195,No,68,no,yes,34,160.0,72,27.20,184.5,119,15.68,208.3,101,9.37,6.1,10,1.65,1


In [4]:
print(df.shape)
df.isnull().sum()

(5000, 18)


churn                         0
accountlength                 0
internationalplan             0
voicemailplan                 0
numbervmailmessages           0
totaldayminutes               0
totaldaycalls                 0
totaldaycharge                0
totaleveminutes               0
totalevecalls                 0
totalevecharge                0
totalnightminutes             0
totalnightcalls               0
totalnightcharge              0
totalintlminutes              0
totalintlcalls                0
totalintlcharge               0
numbercustomerservicecalls    0
dtype: int64

In [5]:
print(df.dtypes)
df.nunique()

churn                          object
accountlength                   int64
internationalplan              object
voicemailplan                  object
numbervmailmessages             int64
totaldayminutes               float64
totaldaycalls                   int64
totaldaycharge                float64
totaleveminutes               float64
totalevecalls                   int64
totalevecharge                float64
totalnightminutes             float64
totalnightcalls                 int64
totalnightcharge              float64
totalintlminutes              float64
totalintlcalls                  int64
totalintlcharge               float64
numbercustomerservicecalls      int64
dtype: object


churn                            2
accountlength                  218
internationalplan                2
voicemailplan                    2
numbervmailmessages             48
totaldayminutes               1961
totaldaycalls                  123
totaldaycharge                1961
totaleveminutes               1879
totalevecalls                  126
totalevecharge                1659
totalnightminutes             1853
totalnightcalls                131
totalnightcharge              1028
totalintlminutes               170
totalintlcalls                  21
totalintlcharge                170
numbercustomerservicecalls      10
dtype: int64

In [6]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

numerics = ['int64','float64','int32','int16']
catDF = df.select_dtypes(exclude=numerics)
numDF = df.select_dtypes(include=numerics)
print(catDF.head())
numDF.head()

  churn internationalplan voicemailplan
0    No                no           yes
1    No                no           yes
2    No                no            no
3    No               yes            no
4    No               yes            no


,accountlength,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3


In [7]:
numDF = pd.DataFrame(scaler.fit_transform(numDF.values),columns=numDF.columns,index=numDF.index)
numDF.head()

,accountlength,numbervmailmessages,totaldayminutes,totaldaycalls,totaldaycharge,totaleveminutes,totalevecalls,totalevecharge,totalnightminutes,totalnightcalls,totalnightcharge,totalintlminutes,totalintlcalls,totalintlcharge,numbercustomerservicecalls
0,0.518519,1.470588,1.172414,0.384615,1.172890,-0.053156,-0.037037,-0.053819,0.653392,-0.346154,0.652459,-0.085714,-0.333333,-0.085106,0.0
1,0.129630,1.529412,-0.255172,0.884615,-0.255682,-0.081211,0.111111,-0.081597,0.796460,0.115385,0.796721,0.971429,-0.333333,0.978723,0.0
2,0.685185,0.000000,0.873103,0.538462,0.873377,-1.178295,0.370370,-1.178819,-0.557522,0.153846,-0.557377,0.542857,0.333333,0.542553,-1.0
3,-0.296296,0.000000,1.645517,-1.115385,1.646104,-2.053894,-0.444444,-2.053819,-0.051622,-0.423077,-0.052459,-1.057143,1.000000,-1.063830,1.0
4,-0.462963,0.000000,-0.184828,0.500000,-0.185065,-0.778147,0.814815,-0.777778,-0.199115,0.807692,-0.200000,-0.057143,-0.333333,-0.053191,2.0


In [8]:
catDF.drop(['churn'],axis=1,inplace=True)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [9]:
catDF = pd.get_dummies(catDF)
catDF.shape

(5000, 4)

In [10]:
X = pd.concat([catDF,numDF],axis=1)
print(X.shape)

(5000, 19)


In [11]:
Y = df['churn']
Y = Y.replace(to_replace=['No','Yes'],value=[0,1])
print(Y.shape)
Y.sample(15)

(5000,)


793     0
1427    0
629     1
575     0
4254    0
4249    0
105     0
1791    0
1073    0
4835    0
4779    0
31      0
723     0
292     0
4466    0
Name: churn, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
# Using train_test_split to Split Data into Training and Testing Data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100, stratify=Y)

In [20]:
print("Train/Test Sizes : ", X_train.shape, X_test.shape, y_train.shape, y_test.shape, "\n")

Train/Test Sizes :  (3500, 19) (1500, 19) (3500,) (1500,) 



In [21]:
dmat_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)
dmat_test = xgb.DMatrix(X_test, y_test, feature_names=X_train.columns)

In [22]:
booster = xgb.train({'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'},
                    dmat_train,
                    evals=[(dmat_train, "train"), (dmat_test, "test")])

[0]	train-logloss:0.34271	test-logloss:0.34126
[1]	train-logloss:0.26985	test-logloss:0.27977
[2]	train-logloss:0.24852	test-logloss:0.26891
[3]	train-logloss:0.22419	test-logloss:0.25419
[4]	train-logloss:0.19185	test-logloss:0.22256
[5]	train-logloss:0.18572	test-logloss:0.22159
[6]	train-logloss:0.18029	test-logloss:0.22063
[7]	train-logloss:0.17518	test-logloss:0.22068
[8]	train-logloss:0.17243	test-logloss:0.22011
[9]	train-logloss:0.16882	test-logloss:0.21979


In [23]:
print("\nTrain RMSE : ",booster.eval(dmat_train))
print("Test  RMSE : ",booster.eval(dmat_test))


Train RMSE :  [0]	eval-logloss:0.16882119385154717
Test  RMSE :  [0]	eval-logloss:0.21979262319956130


In [24]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

train_preds = [1 if pred>0.5 else 0 for pred in booster.predict(data=dmat_train)]
test_preds = [1 if pred>0.5 else 0 for pred in booster.predict(data=dmat_test)]

print("\nTest  Accuracy : %.2f"%accuracy_score(y_test, test_preds))
print("Train Accuracy : %.2f"%accuracy_score(y_train, train_preds))

print("\nConfusion Matrix : ")
print(confusion_matrix(y_test, test_preds))

print("\nClassification Report : ")
print(classification_report(y_test, test_preds))


Test  Accuracy : 0.92
Train Accuracy : 0.94

Confusion Matrix : 
[[1253   35]
 [  79  133]]

Classification Report : 
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      1288
           1       0.79      0.63      0.70       212

    accuracy                           0.92      1500
   macro avg       0.87      0.80      0.83      1500
weighted avg       0.92      0.92      0.92      1500



In [25]:
# Save the model
booster.save_model("binary_classification.model")

In [26]:
loaded_booster =  xgb.Booster()
loaded_booster

In [27]:
loaded_booster.load_model("binary_classification.model")

In [28]:
pd.DataFrame({"Preds":booster.predict(dmat_test)[:5], "Loaded Model Preds":loaded_booster.predict(dmat_test)[:5]})

,Preds,Loaded Model Preds
0,0.020981,0.020981
1,0.031051,0.031051
2,0.020981,0.020981
3,0.021110,0.021110
4,0.055564,0.055564


In [29]:
loaded_booster1 =  xgb.Booster(model_file="binary_classification.model")


In [30]:
pd.DataFrame({"Preds":booster.predict(dmat_test)[:5], "Loaded Model Preds":loaded_booster1.predict(dmat_test)[:5]})

,Preds,Loaded Model Preds
0,0.020981,0.020981
1,0.031051,0.031051
2,0.020981,0.020981
3,0.021110,0.021110
4,0.055564,0.055564


In [31]:
dmat_train = xgb.DMatrix(X_train,y_train,feature_names=X_train.columns)
xgb.cv({'max_depth':5,'eta':1,'objective':'binary:logistic'},dmat_train,num_boost_round=10,nfold=5)

,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.222575,0.006284,0.249955,0.020758
1,0.147789,0.005238,0.182055,0.027772
2,0.118801,0.007364,0.161531,0.029363
3,0.104242,0.006418,0.160535,0.035573
4,0.092362,0.005458,0.164967,0.033707
5,0.080735,0.004172,0.169642,0.039951
6,0.073261,0.005145,0.172131,0.041445
7,0.067678,0.005072,0.173531,0.044520
8,0.063587,0.006049,0.174416,0.043132
9,0.057254,0.005073,0.177878,0.043921


In [32]:
from sklearn.model_selection import KFold, ShuffleSplit

shuffle_split = ShuffleSplit(random_state=123)

dmat_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

xgb.cv({'max_depth': 5, 'eta': 1, 'objective': 'binary:logitraw'}, dmat_train, folds=shuffle_split)

[12:53:37] WARNING: ../src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:53:37] WARNING: ../src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:53:37] WARNING: ../src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:53:37] WARNING: ../src/learner.cc:1205: Starting in XGBoost 1.4.0, the default evaluation metric used with the objective 'binary:logitraw' was changed from 'auc' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,train-logloss-mean,train-logloss-std,test-logloss-mean,test-logloss-std
0,0.394531,0.095229,0.826120,0.265722
1,-0.114386,0.133364,0.354541,0.400458
2,-0.356183,0.074080,0.167483,0.343033
3,-0.535931,0.038620,-0.008310,0.187124
4,-0.637132,0.060019,-0.149812,0.245004
5,-0.716763,0.040388,-0.104009,0.290026
6,-0.788227,0.064257,-0.080625,0.274950
7,-0.851666,0.069659,-0.038485,0.266133
8,-0.918276,0.049580,-0.012382,0.209805
9,-1.021085,0.063600,-0.033809,0.281590


In [33]:
from sklearn.model_selection import KFold, ShuffleSplit

shuffle_split = ShuffleSplit(random_state=123)

dmat_train = xgb.DMatrix(X_train, y_train, feature_names=X_train.columns)

xgb.cv({'max_depth': 5, 'eta': 1, 'objective': 'binary:logitraw'}, dmat_train, folds=shuffle_split, metrics=['auc','logloss','error'])

,train-auc-mean,train-auc-std,train-logloss-mean,train-logloss-std,train-error-mean,train-error-std,test-auc-mean,test-auc-std,test-logloss-mean,test-logloss-std,test-error-mean,test-error-std
0,0.913856,0.006185,0.394531,0.095229,0.045206,0.002393,0.904635,0.023537,0.826120,0.265722,0.059143,0.014174
1,0.932146,0.003954,-0.114386,0.133364,0.038571,0.003082,0.924719,0.021665,0.354541,0.400458,0.053714,0.008061
2,0.952421,0.004676,-0.356183,0.074080,0.031619,0.001524,0.933663,0.020008,0.167483,0.343033,0.046571,0.007565
3,0.968105,0.004250,-0.535931,0.038620,0.028571,0.001581,0.934729,0.026020,-0.008310,0.187124,0.045429,0.005918
4,0.976799,0.004429,-0.637132,0.060019,0.026286,0.001360,0.935141,0.026383,-0.149812,0.245004,0.042857,0.006881
5,0.983341,0.003777,-0.716763,0.040388,0.024444,0.001739,0.937988,0.026736,-0.104009,0.290026,0.044571,0.005743
6,0.986760,0.003269,-0.788227,0.064257,0.023429,0.002056,0.936364,0.024494,-0.080625,0.274950,0.045714,0.007667
7,0.990664,0.002607,-0.851666,0.069659,0.021365,0.001779,0.934440,0.021526,-0.038485,0.266133,0.046286,0.005237
8,0.993290,0.001896,-0.918276,0.049580,0.020762,0.001074,0.933892,0.020989,-0.012382,0.209805,0.046571,0.008857
9,0.995811,0.001348,-1.021085,0.063600,0.019079,0.000957,0.931734,0.023411,-0.033809,0.281590,0.049429,0.008576


In [ ]:
%%time

from sklearn.model_selection import GridSearchCV

params = {
        'n_estimators': [50,100,150,200,300,500],
        'max_depth': [None, 3, 5, 7, 9],
        'eta': [0.5, 1, 2, 3]
        }
grid_search = GridSearchCV(xgb.XGBClassifier(), params, n_jobs=-1, cv=5)

grid_search.fit(X_train, y_train)

print("Test  Accuracy Score : %.2f"%grid_search.score(X_test, y_test))
print("Train Accuracy Score : %.2f"%grid_search.score(X_train, y_train))

print("Best Params : ", grid_search.best_params_)
print("Feature Importances : ")
pd.DataFrame([grid_search.best_estimator_.feature_importances_], columns=X_train.columns)